# TP Intelligence Artificielle - Recherche Arborescente Non Informée
# 人工智能实验 - 无信息树搜索

# **Partie 0 : Visualisation des états**
# **第 0 部分：状态可视化**

## Voici une fonction pour visualiser les états. Nous l'utiliserons plus tard.
## 这是一个可视化状态的函数。我们稍后会用到它。

In [1]:
from IPython.display import display, HTML

def visualize_state(state):
    """Visualizes the given state of the Taquin using HTML."""
    """使用 HTML 可视化给定的推盘游戏状态。"""
    html = "<table>"
    for row in state:
        html += "<tr>"
        for tile in row:
            if tile == 0:
                html += "<td style='background-color: lightgray; width: 30px; height: 30px; text-align: center; font-size: 20px;'> </td>"  # Blank tile
            else:
                html += f"<td style='background-color: blue; width: 30px; height: 30px; text-align: center; font-size: 20px;'>{tile}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

# **Partie 1 : Modélisation**
# **第 1 部分：建模**

### Nous allons créer deux classes pour modeliser le taquin en espace d'états.
### 我们将创建两个类来在状态空间中为推盘游戏建模。

1. **Taquin** : Cette classe représente le problème du jeu du Taquin. Elle contient :
1. **Taquin**：此类代表推盘游戏问题。它包含：

  * Attributs:

      * initial_state : L'état initial du Taquin, représenté par une liste de listes. Chaque sous-liste représente une ligne du Taquin, et chaque élément de la sous-liste représente une tuile. La tuile vide est représentée par le chiffre 0.
      * initial_state：推盘游戏的初始状态，由列表的列表表示。每个子列表代表推盘游戏的一行，子列表的每个元素代表一个图块。空图块由数字 0 表示。
      * goal_state : L'état but du Taquin, représenté de la même manière que l'état initial.
      * goal_state：推盘游戏的目标状态，以与初始状态相同的方式表示。
      * size : La taille du Taquin (par exemple, 3 pour un Taquin 3x3, 4 pour un Taquin 4x4).
      * size：推盘游戏的大小（例如，3 代表 3x3 推盘游戏，4 代表 4x4 推盘游戏）。

  * Méthodes:

      * actions(state) : Cette méthode prend un état du Taquin en entrée et retourne une liste des actions possibles à partir de cet état. Les actions possibles sont "haut", "bas", "gauche" et "droite", représentant les mouvements possibles de la tuile vide.
      * actions(state)：此方法将推盘游戏的状态作为输入，并返回从该状态开始的可能操作列表。可能的操作是“上”、“下”、“左”和“右”，代表空图块的可能移动。
      * result(state, action) : Cette méthode prend un état et une action en entrée et retourne le nouvel état du Taquin après avoir appliqué l'action à l'état.
      * result(state, action)：此方法将状态和操作作为输入，并返回将操作应用于状态后的推盘游戏新状态。
      * is_goal(state) : Cette méthode prend un état en entrée et retourne True si cet état est l'état but, False sinon.
      * is_goal(state)：此方法将状态作为输入，如果此状态是目标状态，则返回 True，否则返回 False。
      * cost(state, action) : Cette méthode retourne le coût de l'application d'une action à un état donné. Dans le cas du Taquin, le coût est généralement constant et égal à 1 pour chaque action.
      * cost(state, action)：此方法返回将操作应用于给定状态的成本。在推盘游戏的情况下，成本通常是恒定的，每个操作的成本等于 1。
  
2.   **Node** :  Cette classe représentera un nœud dans l'arbre de recherche. Ces attributs sont :
2.   **Node**：此类将代表搜索树中的一个节点。这些属性是：

  * state : L'état représenté par ce nœud.
  * state：此节点代表的状态。
  * parent : Un pointeur vers le nœud parent (None pour le nœud racine).
  * parent：指向父节点的指针（根节点为 None）。
  * action : L'action qui a conduit à ce nœud à partir du nœud parent (None pour le nœud racine).
  * action：导致从父节点到达此节点的操作（根节点为 None）。
  * path_cost : Le coût total du chemin depuis le nœud racine jusqu'à ce nœud (facultatif, pour les algorithmes avec des considérations de coût).
  * path_cost：从根节点到此节点的路径总成本（可选，用于考虑成本的算法）。
  * depth : La profondeur de ce nœud dans l'arbre (facultatif, pour la recherche en profondeur limitée).
  * depth：此节点在树中的深度（可选，用于有限深度搜索）。



--------------------------------------------------------------------------------
## **1.1 Classe Taquin**
--------------------------------------------------------------------------------

In [2]:
class Taquin:
    """
    A class representing the Taquin problem.
    代表推盘游戏问题的类。
    """

    def __init__(self, initial_state, goal_state, size):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.size = size

    def actions(self, state):
        """Returns the possible actions (moves) from the given state."""
        """返回从给定状态可能的动作（移动）。"""
        # Find the position of the blank tile (0)
        # 找到空图块 (0) 的位置
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # Define possible moves (up, down, left, right)
        # 定义可能的动作（上、下、左、右）
        possible_actions = []
        if row > 0:
            possible_actions.append("up")
        if row < self.size-1:
            possible_actions.append("down")
        if col > 0:
            possible_actions.append("left")
        if col < self.size-1:
            possible_actions.append("right")

        return possible_actions

    def result(self, state, action):
        """Returns the state that results from applying the given action."""
        """返回应用给定动作后产生的结果状态。"""
        # Create a copy of the state to avoid modifying the original
        # 创建状态的副本以避免修改原始状态
        new_state = [list(row) for row in state]

        # Find the position of the blank tile (0)
        # 找到空图块 (0) 的位置
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # Apply the action to move the blank tile
        # 应用动作以移动空图块
        if action == "up":
            new_state[row][col], new_state[row - 1][col] = (
                new_state[row - 1][col],
                new_state[row][col],
            )
        elif action == "down":
            new_state[row][col], new_state[row + 1][col] = (
                new_state[row + 1][col],
                new_state[row][col],
            )
        elif action == "left":
            new_state[row][col], new_state[row][col - 1] = (
                new_state[row][col - 1],
                new_state[row][col],
            )
        elif action == "right":
            new_state[row][col], new_state[row][col + 1] = (
                new_state[row][col + 1],
                new_state[row][col],
            )

        return new_state

    def is_goal(self, state):
        return state == self.goal_state  # Directly compare with goal_state

    def cost(self, state, action):
        return 1  # Default cost is 1

## **Exercise 1**
## **练习 1**
1. Créez un jeu de taquin 4x4 avec un état initial et un état objectif, puis visualisez ces deux états.
    - 创建一个具有初始状态和目标状态的 4x4 推盘游戏，然后可视化这两个状态。
2. Identifiez les actions possibles à partir de l'état initial.
    - 确定从初始状态开始的可能操作。
3. Appliquez une des actions possibles et visualisez le nouvel état.
    - 应用其中一个可能的操作并可视化新状态。

In [3]:
# 1. 创建一个 4x4 推盘游戏
# 定义初始状态 (注意：0 代表空白格)
initial_state_4x4 = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 0, 15]  # 这里 0 在第三行第三列
]

# 定义目标状态 (通常是顺序排列，0 在最后)
goal_state_4x4 = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 0]
]

# 实例化 Taquin 类，大小为 4
taquin_4x4 = Taquin(initial_state_4x4, goal_state_4x4, size=4)

print("--- 1. 可视化初始状态和目标状态 ---")
print("初始状态:")
visualize_state(taquin_4x4.initial_state)
print("目标状态:")
visualize_state(taquin_4x4.goal_state)

# 2. 确定从初始状态开始的可能操作
actions = taquin_4x4.actions(taquin_4x4.initial_state)
print("\n--- 2. 可能的操作 ---")
print(f"可能的移动方向: {actions}")

# 3. 应用其中一个操作并可视化
# 我们选择列表中的第一个操作来演示
chosen_action = actions[1]
new_state = taquin_4x4.result(taquin_4x4.initial_state, chosen_action)

print(f"\n--- 3. 执行操作 '{chosen_action}' 后的新状态 ---")
visualize_state(new_state)

--- 1. 可视化初始状态和目标状态 ---
初始状态:


1,2,3,4
5,6,7,8
9,10,11,12
13,14,,15


目标状态:


1,2,3,4
5,6,7,8
9,10,11,12
13,14,15,



--- 2. 可能的操作 ---
可能的移动方向: ['up', 'left', 'right']

--- 3. 执行操作 'left' 后的新状态 ---


1,2,3,4
5,6,7,8
9,10,11,12
13,,14,15


--------------------------------------------------------------------------------
## **1.2 Classe Node**
--------------------------------------------------------------------------------

In [ ]:
class Node:
    """
    A node in a search tree.
    搜索树中的一个节点。
    __init__: Initializes a node with its state, parent, action, path cost, and depth.
    __init__：使用其状态、父节点、动作、路径成本和深度初始化节点。
    __repr__: Provides a string representation of the node.
    __lt__: Defines a comparison operator for nodes based on their states.
    expand: Generates child nodes by applying all possible actions.
    child_node: Creates a single child node for a given action.
    solution: Returns the sequence of actions that led to this node.
    path: Returns the path from the root to this node as a list of nodes.
    path：返回从根节点到此节点的路径（节点列表）。
    """

    def __init__(self, state, parent=None, action=None, path_cost = 0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0 if parent is None else parent.depth + 1

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        """列出从此节点一步可达的节点。"""
        return [
            self.child_node(problem, action)
            for action in problem.actions(self.state)
        ]

    def child_node(self, problem, action):
        """Create a child node by applying the given action."""
        """通过应用给定的动作创建一个子节点。"""
        next_state = problem.result(self.state, action)
        next_node = Node(
            next_state,
            parent=self,
            action=action,
            path_cost = self.path_cost + problem.cost(self.state, action),
        )
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        """返回从根节点到此节点的动作序列。"""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        """返回形成从根节点到此节点路径的节点列表。"""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

## **Exercise 2** 
## **练习 2**
1. Créez un jeu de taquin 3x3 avec un état initial et un état objectif.
    - 创建一个具有初始状态和目标状态的 3x3 推盘游戏。
2. Créez un nœud représentant l'état initial du jeu.
    - 创建一个代表游戏初始状态的节点。
3. Visualisez les enfants de l'état initial.
    - 可视化初始状态的子节点。
4. Pour chaque enfant, imprimez :
    - 对于每个子节点，打印：
        * L'action à effectuer pour passer de l'état initial à cet enfant.
            - 从初始状态移动到该子节点所执行的动作。
        * Le coût associé à cette action.
            - 与此动作关联的成本。
5. Répétez toutes les étapes pour un jeu de taquin 4x4.
    - 对 4x4 推盘游戏重复所有步骤。

In [5]:
# ==========================================
# 第一部分：3x3 推盘游戏
# ==========================================
print("=== 1. Jeu de Taquin 3x3 (3x3 推盘游戏) ===")

# 1. 创建 3x3 游戏实例
initial_state_3x3 = [
    [1, 2, 3],
    [4, 0, 6],  # 0 在中间
    [7, 5, 8]
]
goal_state_3x3 = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]
problem_3x3 = Taquin(initial_state_3x3, goal_state_3x3, size=3)

# 2. 创建代表初始状态的节点 (根节点)
root_node_3x3 = Node(problem_3x3.initial_state)

# 3. & 4. 扩展当前节点，获取所有子节点，并打印动作、成本和可视化
# expand() 方法会自动调用 actions() 和 child_node()
children_3x3 = root_node_3x3.expand(problem_3x3)

print("\n--- Enfants de l'état initial (初始状态的子节点) ---")
for i, child in enumerate(children_3x3):
    print(f"\n子节点 {i+1}:")
    print(f"  - Action (执行的动作): {child.action}")
    print(f"  - Coût (成本): {problem_3x3.cost(root_node_3x3.state, child.action)}")
    visualize_state(child.state)


# ==========================================
# 第二部分：4x4 推盘游戏
# ==========================================
print("\n\n=== 5. Jeu de Taquin 4x4 (4x4 推盘游戏) ===")

# 重复上述步骤
initial_state_4x4 = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 0, 15] 
]
goal_state_4x4 = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 0]
]

problem_4x4 = Taquin(initial_state_4x4, goal_state_4x4, size=4)
root_node_4x4 = Node(problem_4x4.initial_state)

# 扩展并可视化
children_4x4 = root_node_4x4.expand(problem_4x4)

print("\n--- Enfants de l'état initial 4x4 (4x4 初始状态的子节点) ---")
for i, child in enumerate(children_4x4):
    print(f"\n子节点 {i+1}:")
    print(f"  - Action (执行的动作): {child.action}")
    print(f"  - Coût (成本): {problem_4x4.cost(root_node_4x4.state, child.action)}")
    visualize_state(child.state)

=== 1. Jeu de Taquin 3x3 (3x3 推盘游戏) ===

--- Enfants de l'état initial (初始状态的子节点) ---

子节点 1:
  - Action (执行的动作): up
  - Coût (成本): 1


1,,3
4,2,6
7,5,8



子节点 2:
  - Action (执行的动作): down
  - Coût (成本): 1


1,2,3
4,5,6
7,,8



子节点 3:
  - Action (执行的动作): left
  - Coût (成本): 1


...

1,2,3,4
5,6,7,8
9,10,,12
13,14,11,15



子节点 2:
  - Action (执行的动作): left
  - Coût (成本): 1


1,2,3,4
5,6,7,8
9,10,11,12
13,,14,15



子节点 3:
  - Action (执行的动作): right
  - Coût (成本): 1


1,2,3,4
5,6,7,8
9,10,11,12
13,14,15,


# **Partie 2 : BFS et DFS**
# **第 2 部分：BFS 和 DFS**


Dans cette 2ème partie nous allons coder les algorithmes de recherche arborescente que nous avons étudiés en cours.
    在这一部分中，我们将编写我们在课程中学习的树搜索算法。

## **2.1 Breadth First Search**
## **2.1 广度优先搜索**


### **Exercice 3** 

Créer une fonction BFS qui prend un objet problem comme entrée et exécute l'algorithme de recherche en profondeur. Votre fonction doit :
Créer une fonction BFS qui prend un objet problem comme entrée et exécute l'algorithme de recherche en profondeur. Votre fonction doit :
    创建一个 BFS 函数，该函数以 problem 对象作为输入并执行广度优先搜索算法。您的函数必须：

1. Retourner le nœud objectif trouvé ainsi que le nombre de nœuds explorés.
    - 返回找到的目标节点以及探索的节点数。
2. Retourner None si l'algorithme explore tout l'arbre sans trouver le nœud objectif.
    - 如果算法探索了整棵树但没有找到目标节点，则返回 None。
3. Prendre en compte un budget d'exploration pour arrêter l'algorithme si aucune solution n'est trouvée après plusieurs itérations (utile pour les grands problèmes). Le budget sera également une entrée de la fonction, avec float('inf') comme valeur par défaut.
    - 考虑探索预算，如果在多次迭代后未找到解，则停止算法（对于大型问题很有用）。预算也将是函数的输入，默认值为 float('inf')。

Pour la frontière et l'ensemble des nœuds déjà explorés :
对于边界和已探索节点集合：
* La frontière doit être une liste de nœuds (objets créés par la classe Node).
    - 边界必须是节点列表（由 Node 类创建的对象）。
* L'ensemble des nœuds explorés peut être une liste d'états.
    - 已探索节点集合可以是状态列表。
* Pour sélectionner un élément de la frontière à explorer, utilisez la méthode *pop*. Assurez-vous de toujours prendre le premier élément de la liste.
    - 要选择要探索的边界元素，请使用 *pop* 方法。确保总是取出列表中的第一个元素。

Pour ajouter un élément à une liste, utilisez *liste.append(element)*.

In [6]:
def BFS(problem, budget=float('inf')):
    """
    Breadth-First Search (BFS) 算法
    :param problem: 问题对象 (Taquin)
    :param budget: 探索预算 (最大节点数)
    :return: (found_node, num_explored) 或 None
    """
    
    # 1. 创建初始节点
    node = Node(problem.initial_state)
    
    # 特殊情况：如果初始状态就是目标
    if problem.is_goal(node.state):
        return node, 0

    # 2. 初始化边界 (Frontier) - 作为队列使用
    frontier = [node]
    
    # 3. 初始化已探索状态集合 (Explored)
    explored = []
    
    # 计数：探索过的节点数量
    num_explored = 0

    # 4. 主循环
    while frontier:
        # 如果超出预算，停止
        if num_explored >= budget:
            return None

        # 从边界的【最前面】取出一个节点 (FIFO: pop(0))
        node = frontier.pop(0)
        num_explored += 1
        
        # 将当前状态加入已探索列表
        explored.append(node.state)

        # 扩展当前节点，获取所有子节点
        for child in node.expand(problem):
            # 检查子节点的状态是否已经在 'explored' 或 'frontier' 中
            # (注意：因为状态是列表，不能用 set 快速查找，只能遍历检查)
            state_in_frontier = any(child.state == n.state for n in frontier)
            
            if child.state not in explored and not state_in_frontier:
                # 在加入队列前检查是否是目标 (这是 BFS 的标准优化)
                if problem.is_goal(child.state):
                    return child, num_explored
                
                # 如果不是目标，加入边界队列的末尾
                frontier.append(child)
    
    # 如果队列空了还没找到
    return None

### **Exercice 4**
### **练习 4**
1. Appliquez votre fonction BFS pour résoudre un Taquin 3x3 avec l'état initial
1. 应用您的 BFS 函数解决具有初始状态的 3x3 推盘游戏
   [[1, 2, 3], [4, 5, 6], [0, 7, 8]].
   Imprimez :
   打印：
    * Le nombre de nœuds explorés par l'algorithme.
    * 算法探索的节点数。
    * Le chemin pour passer de l'état initial à l'état objectif (la liste des actions effectuées).
    * 从初始状态到目标状态的路径（执行的动作列表）。
    * Les états successifs du chemin (utilisez la fonction visualize_state).
    * 路径的连续状态（使用 visualize_state 函数）。
3. Répétez la même expérience pour l'état initial [[1, 2, 3], [4, 5, 0], [6, 7, 8]].
3. 对初始状态 [[1, 2, 3], [4, 5, 0], [6, 7, 8]] 重复相同的实验。

In [ ]:
# ==========================================
# 辅助函数：专门用来打印结果和展示步骤
# ==========================================
def run_and_display_bfs(initial_state, goal_state, case_name):
    print(f"\n{'='*20} {case_name} {'='*20}")
    
    # 1. 定义问题
    problem = Taquin(initial_state, goal_state, size=3)
    
    # 2. 运行 BFS
    print("正在搜索解 (BFS)...")
    result = BFS(problem)
    
    if result is not None:
        node, num_explored = result
        print("✅ 找到解了！")
        print(f"📊 探索节点数: {num_explored}")
        
        # 获取动作路径
        actions = node.solution()
        print(f"👣 移动步骤 ({len(actions)}步): {actions}")
        
        # 3. 可视化每一步
        print("\n--- 步骤演示 ---")
        path_nodes = node.path() # 获取从根到叶子的所有节点
        for i, n in enumerate(path_nodes):
            if i == 0:
                print("开始 (Start):")
            else:
                print(f"第 {i} 步: {n.action}")
            visualize_state(n.state)
    else:
        print("❌ 未找到解 (或者超过了预算)。")

# ==========================================
# 测试用例 1
# ==========================================
initial_state_1 = [
    [1, 2, 3],
    [4, 5, 6],
    [0, 7, 8]
]
goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]
run_and_display_bfs(initial_state_1, goal_state, "测试用例 1")

# ==========================================
# 测试用例 2
# ==========================================
initial_state_2 = [
    [1, 2, 3],
    [4, 5, 0],
    [6, 7, 8]
]
# 目标状态是一样的，不用重新定义
run_and_display_bfs(initial_state_2, goal_state, "测试用例 2")


==================== 测试用例 1 ====================
正在搜索解 (BFS)...
✅ 找到解了！
📊 探索节点数: 3
👣 移动步骤 (2步): ['right', 'right']

--- 步骤演示 ---
开始 (Start):


1,2,3
4,5,6
,7,8


第 1 步: right


1,2,3
4,5,6
7,,8


第 2 步: right


...

1,2,3
4,5,6
,7,8


第 12 步: right


1,2,3
4,5,6
7,,8


第 13 步: right


1,2,3
4,5,6
7,8,


## **2.2 Depth First Search**
## **2.2 深度优先搜索**

### **Exercice 5**
### **练习 5**
Créer une fonction DFS qui prend un objet problem comme entrée et exécute l'algorithme de recherche en profondeur. Votre fonction doit :
创建 DFS 函数，输入 problem 对象并执行深度优先搜索算法。您的函数必须：

1. Retourner le nœud objectif trouvé ainsi que le nombre de nœuds explorés.
    - 返回找到的目标节点以及探索的节点数。
2. Retourner None si l'algorithme explore tout l'arbre sans trouver le nœud objectif.
    - 如果算法探索了整棵树但没有找到目标节点，则返回 None。
3. Prendre en compte un budget d'exploration pour arrêter l'algorithme si aucune solution n'est trouvée après plusieurs itérations (utile pour les grands problèmes). Le budget sera également une entrée de la fonction, avec float('inf') comme valeur par défaut.
    - 考虑探索预算，如果在多次迭代后未找到解，则停止算法（对于大型问题很有用）。预算也将是函数的输入，默认值为 float('inf')。

Pour la frontière et l'ensemble des nœuds déjà explorés :
对于边界和已探索节点集合：
* La frontière doit être une liste de nœuds (objets créés par la classe Node).
* 边界必须是节点列表（由 Node 类创建的对象）。
* L'ensemble des nœuds explorés peut être une liste d'états.
* 已探索节点集合可以是状态列表。
* Pour sélectionner un élément de la frontière à explorer, utilisez la méthode *pop*. Assurez-vous de toujours prendre le dernier élément de la liste.
* 要选择要探索的边界元素，请使用 *pop* 方法。确保总是取出列表的**最后一个**元素。

Pour ajouter un élément à une liste, utilisez *liste.append(element)*.



In [8]:
def DFS(problem, budget=float('inf')):
    """
    Depth-First Search (DFS) 算法
    与 BFS 的唯一核心区别在于：从边界取出节点的方式 (LIFO vs FIFO)。
    :param problem: 问题对象
    :param budget: 探索预算
    :return: (found_node, num_explored) 或 None
    """
    node = Node(problem.initial_state)
    if problem.is_goal(node.state):
        return node, 0

    # 1. 边界 (Frontier) - 这里我们把它当做【栈 (Stack)】使用
    frontier = [node]
    
    explored = []
    num_explored = 0

    while frontier:
        # 预算控制
        if num_explored >= budget:
            return None

        # 2. 核心区别：使用 pop() 不带参数，默认取出【最后一个】元素 (Stack behavior)
        node = frontier.pop() 
        num_explored += 1
        
        explored.append(node.state)

        # 扩展节点
        for child in node.expand(problem):
            # 检查重复：是否在 frontier 或 explored 中
            state_in_frontier = any(child.state == n.state for n in frontier)
            
            if child.state not in explored and not state_in_frontier:
                # 检查目标
                if problem.is_goal(child.state):
                    return child, num_explored
                
                # 加入栈顶 (List 的末尾)
                frontier.append(child)
    
    return None

### **Exercise 6**
### **练习 6**
1. Appliquez votre fonction DFS pour résoudre un Taquin 3x3 avec l'état initial [[1, 2, 3], [4, 5, 6], [0, 7, 8]]. Imprimez :
    - 应用您的 DFS 函数解决具有初始状态 [[1, 2, 3], [4, 5, 6], [0, 7, 8]] 的 3x3 推盘游戏。打印：
        * Le nombre de nœuds explorés par l'algorithme.
        * 算法探索的节点数。
        * Le chemin pour passer de l'état initial à l'état objectif (la liste des actions effectuées).
        * 从初始状态到目标状态的路径（执行的动作列表）。
        * Les états successifs du chemin (utilisez la fonction visualize_state).
        * 路径的连续状态（使用 visualize_state 函数）。
2. Répétez la même expérience pour l'état initial [[1, 2, 3], [4, 5, 0], [6, 7, 8]].
    - 对初始状态 [[1, 2, 3], [4, 5, 0], [6, 7, 8]] 重复相同的实验。
3. Qu'observez-vous en comparant cet algorithme avec BFS ?
    - 比较此算法与 BFS，您观察到了什么？

In [ ]:
# ==========================================
# 辅助函数：运行 DFS 并打印结果
# ==========================================
def run_and_display_dfs(initial_state, goal_state, case_name, budget=3000):
    print(f"\n{'='*20} {case_name} {'='*20}")
    
    # 定义问题
    problem = Taquin(initial_state, goal_state, size=3)
    
    # 运行 DFS (默认限制3000个节点，防止死循环或跑太久)
    print(f"正在搜索解 (DFS) [Budget={budget}]...")
    result = DFS(problem, budget=budget)
    
    if result:
        node, num_explored = result
        print("✅ 找到解了！")
        print(f"📊 探索节点数: {num_explored}")
        
        # 获取路径
        actions = node.solution()
        print(f"👣 路径长度: {len(actions)}")
        print(f"📝 具体步骤 (前20步): {actions[:20]} ... (太长省略)" if len(actions) > 20 else f"📝 具体步骤: {actions}")
        
        # 只可视化最后几步，不然太多了
        print("\n--- 结果演示 (为了不刷屏，只显示 Start 和 End) ---")
        visualize_state(node.path()[0].state) # Start
        print("... (中间省略) ...")
        visualize_state(node.state)           # End
    else:
        print("❌ 未找到解 (已达到搜索预算限制)。")

# ==========================================
# 测试用例 1
# ==========================================
# 这是一个简单的例子，DFS 应该能很快解决，但路径可能不完美
initial_state_1 = [
    [1, 2, 3],
    [4, 5, 6],
    [0, 7, 8]
]
goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]
run_and_display_dfs(initial_state_1, goal_state, "测试用例 1")

# ==========================================
# 测试用例 2
# ==========================================
# 这是一个稍复杂的例子，DFS 可能会陷入深层搜索，找到一个非常绕的解，甚至搜索超时
initial_state_2 = [
    [1, 2, 3],
    [4, 5, 0],
    [6, 7, 8]
]
run_and_display_dfs(initial_state_2, goal_state, "测试用例 2", budget=10000)


==================== 测试用例 1 ====================
正在搜索解 (DFS) [Budget=3000]...
✅ 找到解了！
📊 探索节点数: 2
👣 路径长度: 2
📝 具体步骤: ['right', 'right']

--- 结果演示 (为了不刷屏，只显示 Start 和 End) ---


1,2,3
4,5,6
,7,8


... (中间省略) ...


1,2,3
4,5,6
7,8,



==================== 测试用例 2 ====================
正在搜索解 (DFS) [Budget=10000]...
❌ 未找到解 (已达到搜索预算限制)。


## **2.3 Comparaison empirique de DFS et BFS**
## **2.3 DFS 和 BFS 的经验比较**

Nous allons comparer la performance des deux algorithmes dans plusieurs jeux de Taquin 3x3. 
    我们将比较这两种算法在几个 3x3 推盘游戏中的性能。

### **Exercice 7**
### **练习 7**
1. Écrivez une fonction compare_algorithms qui prend en entrée :
    - 编写一个 compare_algorithms 函数，其输入为：
        * Une liste d’états initiaux.
            * 初始状态列表。
        * Un état objectif.
            * 目标状态。
        * Une liste d’algorithmes.
            * 算法列表。
2. La fonction doit résoudre chaque problème en exécutant chaque algorithme. Assurez-vous de fixer des budgets pour éviter que les algorithmes ne prennent trop de temps.
    - 该函数必须通过执行每个算法来解决每个问题。确保设置预算以防止算法花费太长时间。
3. Pour chaque paire (problème, algorithme), enregistrez :
    - 对于每对（问题，算法），记录：
        * Le nombre de nœuds explorés.
            * 探索的节点数。
        * La longueur du chemin trouvé entre la racine et le nœud objectif.
            * 从根节点到目标节点的路径长度。
        * Le temps d'exécution. Pour mesurer le temps d'exécution, vous pouvez importer time et utiliser :
            * 执行时间。要测量执行时间，您可以导入 time 并使用：
            * *start_time = time.time()*
            * *\# résoudre le problème*
            * *end_time = time.time()*
            * *execution_time = end_time - start_time*
4. Enregistrez les résultats dans un dictionaire.
    - 将结果保存在字典中。



In [10]:
import time

def compare_algorithms(initial_states, goal_state, algorithms, budget=10000):
    """
    比较不同算法在多个初始状态下的表现。
    :param initial_states: 初始状态列表 (List of lists)
    :param goal_state: 目标状态
    :param algorithms: 算法函数列表 [BFS, DFS]
    :param budget: 探索预算
    :return: 一个字典，键是算法名，值是结果列表
    """
    # 初始化结果字典，例如: {'BFS': [], 'DFS': []}
    results = {alg.__name__: [] for alg in algorithms}
    
    # 遍历每一个“考题”（初始状态）
    for i, state in enumerate(initial_states):
        # 创建问题实例 (假设都是 3x3)
        problem = Taquin(state, goal_state, size=3)
        
        # 让每个“考生”（算法）都来做一遍这道题
        for alg in algorithms:
            # 1. 计时开始
            start_time = time.time()
            
            # 2. 运行算法
            result = alg(problem, budget=budget)
            
            # 3. 计时结束
            end_time = time.time()
            execution_time = end_time - start_time
            
            # 4. 记录这一轮的成绩
            if result:
                node, num_explored = result
                path_length = len(node.solution())
            else:
                # 如果没解出来（比如超时），记作失败
                num_explored = budget
                path_length = -1 # 用 -1 代表未找到
            
            # 将成绩存入列表
            results[alg.__name__].append({
                'num_explorations': num_explored,
                'path_length': path_length,
                'execution_time': execution_time
            })
            
    return results

### **Exercice 8**
### **练习 8**
Comparez les algorithmes BFS et DFS dans les instances suivantes du jeu de Taquin 3x3.
在以下 3x3 推盘游戏实例中比较 BFS 和 DFS 算法。
1. [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
2. [[1, 2, 3], [0, 5, 6], [4, 7, 8]]    
3. [[1, 0, 3], [4, 2, 5], [7, 8, 6]]
4. [[1, 0, 3], [4, 5, 2], [7, 6, 8]]
5. [[1, 0, 3], [4, 2, 5], [6, 7, 8]]
6. [[1, 2, 3], [4, 0, 6], [7, 5, 8]]
7. [[1, 2, 3], [0, 4, 6], [7, 5, 8]]
8. [[1, 2, 3], [4, 6, 0], [7, 5, 8]]
9. [[1, 3, 6], [4, 2, 5], [7, 0, 8]]  
10. [[1, 3, 6], [4, 2, 0], [7, 5, 8]]  
11. [[1, 3, 6], [4, 0, 2], [7, 5, 8]]  
12. [[3, 1, 2], [4, 6, 5], [7, 0, 8]]  
13. [[8, 1, 2], [0, 4, 3], [7, 6, 5]]
14. [[1, 4, 2], [7, 0, 6], [5, 3, 8]]
15. [[2, 8, 3], [1, 6, 4], [7, 0, 5]]

In [20]:
# 1. 定义所有待测试的初始状态 (共 15 个)
initial_states = [
    [[1, 2, 3], [4, 5, 0], [6, 7, 8]],
    [[1, 2, 3], [0, 5, 6], [4, 7, 8]],
    [[1, 0, 3], [4, 2, 5], [7, 8, 6]],
    [[1, 0, 3], [4, 5, 2], [7, 6, 8]],
    [[1, 0, 3], [4, 2, 5], [6, 7, 8]],
    [[1, 2, 3], [4, 0, 6], [7, 5, 8]],
    [[1, 2, 3], [0, 4, 6], [7, 5, 8]],
    [[1, 2, 3], [4, 6, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 2, 5], [7, 0, 8]],
    [[1, 3, 6], [4, 2, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 0, 2], [7, 5, 8]],
    [[3, 1, 2], [4, 6, 5], [7, 0, 8]],
    [[8, 1, 2], [0, 4, 3], [7, 6, 5]],
    [[1, 4, 2], [7, 0, 6], [5, 3, 8]],
    [[2, 8, 3], [1, 6, 4], [7, 0, 5]]
]

# 2. 定义目标状态
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# 3. 定义要测试的算法函数列表
algorithms = [BFS, DFS]

# 4. 设置预算 (Budget)
# 注意：DFS 在复杂情况下极易超时，我们给它设个上限防止卡死
budget = 2000

print(f"正在对比 BFS 和 DFS (共 {len(initial_states)} 个测试用例)...")
print("请稍候，这可能需要几秒到几十秒...")

# 5. 调用之前写的对比函数
results = compare_algorithms(initial_states, goal_state, algorithms, budget=budget)

# 临时查看一下数据（确认跑通了）
print("✅ 测试完成！数据已生成。")
print(f"BFS结果数量: {len(results['BFS'])}")
print(f"DFS结果数量: {len(results['DFS'])}")

正在对比 BFS 和 DFS (共 15 个测试用例)...
请稍候，这可能需要几秒到几十秒...
✅ 测试完成！数据已生成。
BFS结果数量: 15
DFS结果数量: 15


#### Utilisez le code ci-dessous pour visualiser vos résultats

In [21]:
import pandas as pd
# 也不需要 numpy

# 如果没有安装 tabulate，可以尝试安装，或者直接用 print(df)
try:
    from tabulate import tabulate
    has_tabulate = True
except ImportError:
    has_tabulate = False

Data = []

# 获取 BFS 和 DFS 的结果列表
bfs_data = results['BFS']
dfs_data = results['DFS']

for i in range(len(initial_states)):
    # 提取第 i 个测试用例的数据
    row = [
        int(i+1), # State 编号
        bfs_data[i]['num_explorations'],
        dfs_data[i]['num_explorations'],
        bfs_data[i]['path_length'],
        dfs_data[i]['path_length'],
        round(bfs_data[i]['execution_time'], 4), # 保留4位小数看更清楚
        round(dfs_data[i]['execution_time'], 4)
    ]
    Data.append(row)

# 创建 DataFrame
df = pd.DataFrame(Data, columns=[
    "State", 
    "Nodes BFS", "Nodes DFS", 
    "Path BFS", "Path DFS", 
    "Time BFS (s)", "Time DFS (s)"
])

# 打印表格
if has_tabulate:
    print(tabulate(df, headers='keys', tablefmt='pretty'))
else:
    print(df) # 如果没有 tabulate 库，直接打印 pandas dataframe 也很清晰
    print(df.mean())

    State  Nodes BFS  Nodes DFS  Path BFS  Path DFS  Time BFS (s)  \
0       1       1665       2000        13        -1        0.2733   
1       2          6         27         3        27        0.0001   
2       3          7       2000         3        -1        0.0001   
3       4       2000       2000        -1        -1        0.4536   
4       5       2000       2000        -1        -1        0.4444   
5       6          3       2000         2        -1        0.0000   
6       7          8       2000         3        -1        0.0001   
7       8          8       2000         3        -1        0.0001   
8       9       2000       2000        -1        -1        0.4542   
9      10         20       2000         5        -1        0.0002   
10     11         62       2000         6        -1        0.0009   
11     12       2000       2000        -1        -1        0.4454   
12     13       2000       2000        -1        -1        0.4374   
13     14       2000       2000   

# **Extra. Tous les jeux de taquin ont une solution ?**

En cours, nous avons dit que l'espace d'état a une taille de 9!. En réalité, seulement la moitié des configurations sont résolvables. Regardez ce [**lien**](https://fr.wikipedia.org/wiki/Taquin#Configurations_solubles_et_insolubles) et la fonction suivante pour plus d'*insights*.

In [ ]:
import random


def is_solvable(puzzle):
    """Check if a 3x3 Taquin puzzle is solvable."""
    flattened = [tile for row in puzzle for tile in row if tile != 0]
    inversions = sum(
        1
        for i in range(len(flattened))
        for j in range(i + 1, len(flattened))
        if flattened[i] > flattened[j]
    )
    return inversions % 2 == 0